In [2]:
import open3d.core as o3c
import numpy as np

# Tensor

Tensor is a "view" of a data Blob with shape, stride, and a data pointer. It is a multidimensional and homogeneous matrix containing elements of single data type. It is used in Open3D to perform numerical operations. It supports GPU operations as well.

## Tensor creation

Tensor can be created from list, numpy array, another tensor. A tensor of specific data type and device can be constructed by passing a ```o3c.Dtype``` and/or ```o3c.Device``` to a constructor. If not passed, the default data type is inferred from the data, and the default device is CPU.
Note that while creating tensor from a list or numpy array, the underlying memory is not shared and a copy is created.

In [16]:
# Tensor from list.
a = o3c.Tensor([0, 1, 2])
print("Created from list:\n{}".format(a))

# Tensor from Numpy.
a = o3c.Tensor(np.array([0, 1, 2]))
print("\nCreated from numpy array:\n{}".format(a))

# Dtype and inferred from list.
a_float = o3c.Tensor([0.0, 1.0, 2.0])
print("\nDefault dtype and device:\n{}".format(a_float))

# Specify dtype.
a = o3c.Tensor(np.array([0, 1, 2]), dtype=o3c.Dtype.Float64)
print("\nSpecified data type:\n{}".format(a))

# Specify device.
a = o3c.Tensor(np.array([0, 1, 2]), device=o3c.Device("CUDA:0"))
print("\nSpecified device:\n{}".format(a))

Created from list:
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7fbfb6898cb0]

Created from numpy array:
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7fbfb3c7e170]

Default dtype and device:
[0.0 1.0 2.0]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x7fbfb3c7c920]

Specified data type:
[0.0 1.0 2.0]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x7fbfb3c7e4b0]


RuntimeError: [1;31m[Open3D ERROR] MemoryManager::GetDeviceMemoryManager: Unimplemented device[0;m

   Tensor can also be created from another tensor by invoking the copy constructor. This is a shallow copy, the data_ptr will be copied but the memory it points to will not be copied.

In [27]:
# Shallow copy constructor.
vals = np.array([1, 2, 3])
src = o3c.Tensor(vals)
dst = src
src[0] += 10

# Changes in one will get reflected in other.
print("Source tensor:\n{}".format(src))
print("\nTarget tensor:\n{}".format(dst))


Source tensor:
[11 2 3]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7fbfb2747aa0]

Target tensor:
[11 2 3]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7fbfb2747aa0]


## Properties of a tensor

In [30]:
vals = np.array((range(24))).reshape(2, 3, 4)
a = o3c.Tensor(vals,
               dtype=o3c.Dtype.Float64,
               device=o3c.Device("CUDA:0"))
print(f"a.shape: {a.shape}")
print(f"a.strides: {a.strides}")
print(f"a.dtype: {a.dtype}")
print(f"a.device: {a.device}")
print(f"a.ndim: {a.ndim}")

RuntimeError: [1;31m[Open3D ERROR] MemoryManager::GetDeviceMemoryManager: Unimplemented device[0;m

## Copy & device transfer
We can transfer tensors across host and multiple devices.

In [35]:
# Host -> Device.
a_cpu = o3c.Tensor([0, 1, 2])
a_gpu = a_cpu.cuda(0)
print(a_gpu)

# Device -> Host.
a_gpu = o3c.Tensor([0, 1, 2], device=o3c.Device("CUDA:0"))
a_cpu = a_gpu.cpu()
print(a_cpu)

# Device -> another Device.
a_gpu_0 = o3c.Tensor([0, 1, 2], device=o3c.Device("CUDA:0"))
a_gpu_1 = a_gpu_0.cuda(1)
print(a_gpu_1)

CPU:0


RuntimeError: [1;31m[Open3D ERROR] CUDA is not available, cannot copy Tensor.[0;m

## Data Types

Open3d defines seven tensor data types.

| Data type                | dtype               |
|--------------------------|---------------------|
| Uninitialized Tensor     | o3c.Dtype.Undefined |
| 32-bit floating point    | o3c.Dtype.Float32   |
| 64-bit floating point    | o3c.Dtype.Float64   |
| 32-bit integer (signed)  | o3c.Dtype.Int32     |
| 64-bit integer (signed)  | o3c.Dtype.Int64     |
| 8-bit integer (unsigned) | o3c.Dtype.UInt8     |
| Boolean                  | o3c.Dtype.Bool      |

### Type casting

In [37]:
# E.g. float -> int
a = o3c.Tensor([0.1, 1.5, 2.7])
b = a.to(o3c.Dtype.Int32)
print(a)
print(b)

[0.1 1.5 2.7]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x7fbfb38ba110]
[0 1 2]
Tensor[shape={3}, stride={1}, Int32, CPU:0, 0x7fbfbab61c40]


In [36]:
# E.g. int -> float
a = o3c.Tensor([1, 2, 3])
b = a.to(o3c.Dtype.Float32)
print(a)
print(b)

[1 2 3]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7fbfb2749b30]
[1.0 2.0 3.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fbfb24863e0]


## Numpy I/O with direct memory map

Tensors created by passing numpy array to the constructor(```o3c.Tensor(np.array(...)```) do not share memory with the numpy aray. To have shared memory, you can use ```o3c.Tensor.from_numpy(...)``` and ```o3c.Tensor.numpy(...)```. Changes in either of them will get reflected in other.

In [47]:
# Using constructor
np_a = np.ones((5,), dtype=np.int32)
o3_a = o3c.Tensor(np_a)
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to numpy array will not reflect as memory is not shared.
np_a[0] += 100
o3_a[1] += 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

np_a: [1 1 1 1 1]
o3_a: [1 1 1 1 1]
Tensor[shape={5}, stride={1}, Int32, CPU:0, 0x7fbfb82dcb30]

np_a: [101   1   1   1   1]
o3_a: [1 201 1 1 1]
Tensor[shape={5}, stride={1}, Int32, CPU:0, 0x7fbfb82dcb30]


In [51]:
# From numpy.
np_a = np.ones((5,), dtype=np.int32)
o3_a = o3c.Tensor.from_numpy(np_a)

# Changes to numpy array reflects on open3d Tensor and vice versa.
np_a[0] += 100
o3_a[1] += 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

np_a: [101 201   1   1   1]
o3_a: [101 201 1 1 1]
Tensor[shape={5}, stride={1}, Int32, CPU:0, 0x7fbfb82dc180]


In [50]:
# To numpy.
o3_a = o3c.Tensor([1, 1, 1, 1, 1], dtype=o3c.Dtype.Int32)
np_a = o3_a.numpy()

# Changes to numpy array reflects on open3d Tensor and vice versa.
np_a[0] += 100
o3_a[1] += 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

# For CUDA Tensor, call cpu() before calling numpy().
o3_a = o3c.Tensor([1, 1, 1, 1, 1], device=o3c.Device("CUDA:0"))
print(f"\no3_a.cpu().numpy(): {o3_a.cpu().numpy()}")

np_a: [101 201   1   1   1]
o3_a: [101 201 1 1 1]
Tensor[shape={5}, stride={1}, Int32, CPU:0, 0x7fbfbab35d90]

o3_a.cpu().numpy(): [1 1 1 1 1]


## PyTorch I/O with DLPack memory map

In [54]:
import torch

# From PyTorch
th_a = torch.ones((5,)).cuda(0)
o3_a = o3c.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

AttributeError: module 'torch.utils' has no attribute 'dlpack'

In [ ]:
# To PyTorch
o3_a = o3d.core.Tensor([1, 1, 1, 1, 1], device=o3d.core.Device("CUDA:0"))
th_a = torch.utils.dlpack.from_dlpack(o3_a.to_dlpack())
o3_a = o3d.core.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

## Binary element-wise operation:

Supported element-wise binary operations are:
1. Add (+)
2. Sub (-)
3. Mul (*)
4. Div (/)
5. Add_ (+=)
6. Sub_ (-=)
7. Mul_ (*=)
8. Div_ (/=)

Operands have to be of same Device, dtype and Broadcast compatible.

In [63]:
a = o3c.Tensor([1, 1, 1], dtype=o3c.Dtype.Float32)
b = o3c.Tensor([2, 2, 2], dtype=o3c.Dtype.Float32)
print("a + b = {}".format(a + b))
print("a - b = {}".format(a - b))
print("a * b = {}".format(a * b))
print("a / b = {}".format(a / b))

a + b = [3.0 3.0 3.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fbfbab61c40]
a - b = [-1.0 -1.0 -1.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fbfbce5b0a0]
a * b = [2.0 2.0 2.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fbfbce57000]
a / b = [0.5 0.5 0.5]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7fbfbce2df00]


Broadcasting follows the same numpy broadcasting rule as given [here](https://numpy.org/doc/stable/user/basics.broadcasting.html).

In [62]:
# Automated broadcasting.
a = o3c.Tensor(np.ones((2, 3)), dtype=o3c.Dtype.Float32)
b = o3c.Tensor(np.ones((3,)), dtype=o3c.Dtype.Float32)
print("a + b = {}".format(a + b))

a + b = [[2.0 2.0 2.0],
 [2.0 2.0 2.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7fbfbd271410]


## Unary element-wise operation: sqrt, sin, cos, ...

In [ ]:
a = o3c.Tensor([4, 9, 16], dtype=o3c.Dtype.Float32)
print(a.sqrt())
print(a.sin())
print(a.cos())

## Reduction: sum, prod, min, max

In [ ]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)
print(a.sum())
print(a.prod())
print(a.min())
print(a.max())

In [ ]:
# With specified dimension
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)

print(a.sum(dim=(0)), "\n")
print(a.sum(dim=(0, 2)), "\n")
print(a.sum(dim=(0, 2), keepdim=True))

## Slicing, indexing, getitem (returns a view), and setitem

In [ ]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)

# Slicing __getitem__
print(a[1:], "\n")

# Indexing __getitem__
print(a[1, 2], "\n")

# Combined __getitem__
print(a[:-1, 0:3:2, 2])

In [ ]:
# Example __setitem__
print(a, "\n")
a[:, :, 2] = 100
print(a)

## Advanced indexing

In [ ]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3c.Tensor(vals)

# __getitem__
print(a[:, [1, 2], [1, 2]])
print(a[1, [[1, 2], [2, 1]], 0:4:2])

# __setitem__
a[:, 0:2, [1, 2]] = o3c.Tensor(np.array([[100, 200], [300, 400]]))
print(a)

## TensorList

In [ ]:
vals = np.array(range(24), dtype = np.float32).reshape((2, 3, 4))

# Empty TensorList
a = o3c.TensorList([3, 4])
print(a)

# TensorList with single Tensor
b = o3c.TensorList([3, 4], size = 1)
print(b)

# TensorList from tensor
c = o3c.TensorList.from_tensor(o3c.Tensor(vals))
print(c)

# TensorList from multiple tensors
d = o3c.TensorList.from_tensors([o3c.Tensor(vals[0]), o3c.Tensor(vals[1])])
print(d)

# Concatenate TensorLists
print(b + c)
print(o3c.TensorList.concat(b, c))

# Append a Tensor to TensorList
d.push_back(o3c.Tensor(vals[0]))
print(d)

# Append a TensorList to another TensorList
d.extend(b)
print(d)


## Logical operations

In [ ]:
a = o3c.Tensor(np.array([True, False, True, False]))
b = o3c.Tensor(np.array([True, True, False, False]))

print(a.logical_and(b))
print(a.logical_or(b))
print(a.logical_xor(b))

## Comparision Operations

In [ ]:
a = o3c.Tensor([0, 1, -1])
b = o3c.Tensor([0, 0, 0])

print(a > b)
print(a >= b)
print(a < b)
print(a <= b)
print(a == b)
print(a != b)

## Nonzero operations

In [ ]:
a = o3c.Tensor([[3, 0, 0], [0, 4, 0], [5, 6, 0]])

print(a.nonzero())
print(a.nonzero(as_tuple = 1))

## Boolean operations

In [ ]:
a = o3c.Tensor([1, -1, -2, -3])

a[a < 0]
print(a[a < 0])

a[a > 0] = 2
print(a)

## Scalar operations

In [ ]:
a = o3c.Tensor.ones((2, 3), dtype = o3c.Dtype.Float32)

print(a.add(1))
print(a + 1)
print(a + True)
print(a - 1)
print(a - True)
print(a * 10)
print(10 / a)

# Inplace
a.add_(1)
print(a)
a.sub_(1)
print(a)

# Shorthand
a += 1
print(a)
a -= 1
print(a)
a *= 10
print(a)
a /= 2
print(a)